## 1. Setup

In [8]:
import os
import glob
import pandas as pd
import kaggle

## 2. Dataset

In [6]:
download_path = "../data/kaggle_dataset"
dataset_slug = "shahp7575/reddit-posts-with-keyword-coffee/"
os.system(f'kaggle datasets download -d {dataset_slug} -p {download_path} --unzip')

Dataset URL: https://www.kaggle.com/datasets/shahp7575/reddit-posts-with-keyword-coffee/versions/
License(s): MIT


100%|██████████| 41.7M/41.7M [00:00<00:00, 48.0MB/s]


0

In [15]:
kaggle_csv = glob.glob(download_path + "/*.csv")[0]
kaggle_csv

'../data/kaggle_dataset/reddit_coffee_scraper_till_2024-11-13 041137.csv'

In [16]:
df_kaggle = pd.read_csv(kaggle_csv)
df_kaggle.shape

(66826, 10)

## 3. Pre-processing